In [1]:
# ! pip install checklist
# ! pip install nltk
# ! pip install pandas
# ! pip install simpletransformers

^C


In [1]:
import nltk
import spacy
# nltk.download('omw-1.4')

In [2]:
import checklist
from checklist.editor import Editor
from checklist.perturb import Perturb

import pandas as pd
import numpy as np
from simpletransformers.classification import ClassificationModel

np.random.seed(42)

editor = Editor()

In [3]:
#load local model
model = ClassificationModel("bert", "outputs", use_cuda = False)


In [4]:
#load subset
subset = pd.read_csv("data\olid-subset-diagnostic-tests.csv", index_col = 0)
subset

,text,labels
id,,
89200,@USER @USER Who the hell does he think he is?,1
71294,#BREAKING. #Greece: Molotov cocktails fly afte...,1
55633,"#OrrinHatch I can’t believe this sexist , clue...",1
16856,@USER @USER I'll use that one the next time im...,1
26072,0-1 lost my acca on the first fucking fight cba,1
...,...,...
80020,#Repost @USER ・・・ We can’t forget to give prop...,0
45518,@USER He is obviously getting suspended. He is...,0
51610,#Canada - EXCLUSIVE: #Trudeau #Liberals leave ...,0


In [5]:
#need spacy model
nlp = spacy.load('en_core_web_sm')

In [6]:
#convert instances to list, then to spacy DOC tokens
data = subset["text"].tolist()
pdataset = list(nlp.pipe(data))
pdataset


[@USER @USER Who the hell does he think he is?,
 #BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL,
 #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!,
 @USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes. Yes you can choose to be irresponsible or choose not to be. I argue responsible. Whats wrong with that? Don't justify murder by saying it was never alive or its my right.,
 0-1 lost my acca on the first fucking fight cba,
 #Bakersfield is why we need gun control! Sorry for the victims other than the gunman himself I hope he rots in hell!,
 #Christian #America – If we go by #Trump’s example, where liberals support open borders, I guess conservatives support

In [55]:
#add negation to sentences
ret = Perturb.perturb(pdataset, Perturb.add_negation, keep_original=True)

In [56]:
#question 2 & 3: look at first 10 examples
ret.data[:10]

[['@USER @USER Who the hell does he think he is?',
  "@USER @USER Who the hell doesn't he think he is?"],
 ['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
  "#BREAKING. #Greece: Molotov cocktails don't fly after protest honouring killed antifa arti... URL via @USER URL"],
 ['#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!',
  '#OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is not spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!'],
 ["@USER @USER I'll use that one the next time im in a gun control debate or in a debate about free speech or taxes.

In [16]:
pred_orig = model.predict([ret.data[0][0]])
pred_perturb = model.predict([ret.data[0][1]])

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/1 [00:00<?, ?it/s]

In [57]:
print(ret.data[0][0])
print(f'original data: {pred_orig}')
print(f'perturbed data: {pred_perturb}')
len(ret.data)

@USER @USER Who the hell does he think he is?
original data: ([1], array([[-3.064749  ,  2.16581225]]))
perturbed data: ([1], array([[-3.01825523,  2.13671899]]))


95

In [33]:
#make prediction for original data and perturbed data to compare
original_predictions=model.predict([sent[0] for ix, sent in enumerate(ret.data)])
perturb_predictions=model.predict([sent[1] for ix, sent in enumerate(ret.data)])

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

  0%|          | 0/95 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

In [52]:
print(perturb_predictions[0])
print(original_predictions[0])

[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [51]:
#check where model assigned different label after perturb
compare_order = [1 if i==j else 0 for i, j in zip(perturb_predictions[0],original_predictions[0])]
print(compare_order)
#only 1 sentence changed label

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [58]:
idx=compare_order.index(0)
print(ret.data[idx][0])
print(ret.data[idx][1])

@USER #MAGA who cares about the farm.  He had no reason to commit murder. End of story.
@USER #MAGA who cares about the farm.  He didn't have no reason to commit murder. End of story.


In [59]:
#get id of tweet
subset.loc[subset['text'].str.contains("cause everyone knows a real leader falls into a van like a sack of potatoes")]

,text,labels
id,,
79934,@USER 'cause everyone knows a real leader fall...,1
